# Naver Cafe Crawler
- Need to crawling 'iframe'
- iframe을 사용하는 장단점
    - 장점 
        - 컨텐츠 추가가 쉬움
        - 페이지 이동이 필요없으므로 응용프로그램을 다루는 느낌을 제공
    - 단점
        - 유지보수가 힘듦
        - 크롤링을 방해하여 노출이 잘 되지 않음
        - 이제는 SPA가 대세이므로 굳이 사용할 필요가 없음

### reference
- chrome driver : https://sites.google.com/a/chromium.org/chromedriver/downloads
- chrome 정보 : chrome://settings/help

In [5]:
!pip install selenium

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [61]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd

In [97]:
# crawling info
# 직접 확인 필요
main_url = 'https://cafe.naver.com/number114?iframe_url='

cat_lst = [
    '중고차 다마스/라보',
    '중고차 1톤~1.4톤',
    '중고차 2.5톤~3.5톤',
    '중고차 4.5톤~8톤 ',
    '중고차 9.5톤~25톤',
    '중고차 탱크로리',
    '중고차 특수차량/기타',
]

iframe_url_lst = [
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=113%26search.boardtype=I%26search.page=', #중고차 다마스/라보
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=14%26search.boardtype=I%26search.page=', #중고차 1톤~1.4톤
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=116%26search.boardtype=I%26search.page=', #중고차 2.5톤~3.5톤 
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=95%26search.boardtype=I%26search.page=', #중고차 4.5톤~8톤 
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=96%26search.boardtype=I%26search.page=', #중고차 9.5톤~25톤
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=118%26search.boardtype=I%26search.page=', #중고차 탱크로리
    '/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=97%26search.boardtype=I%26search.page=', #중고차 특수차량/기타
]

max_page_lst = [1, 16, 13, 33, 18, 1,5]

In [95]:
# start driver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [107]:
result = []

for cat, url, max_page in zip(cat_lst, iframe_url_lst, max_page_lst) :
    print('['+cat+']', ":", main_url+url)
    
    titles = []
    for idx in range(1, max_page+1) :
        # page로 이동
        iframe_page = main_url + url + str(idx)
        driver.get(iframe_page)
        driver.implicitly_wait(3)
        
        # iframe으로 전환
        iframe = driver.find_element_by_id('cafe_main')
        driver.switch_to.frame(iframe)
        
        # html parsing
        html = driver.page_source
        soup = bs(html, 'html.parser')
        temp = [(cat, idx, iframe_page, tag.text.strip(), main_url+tag['href']) for tag in soup.find_all('a', {'class' : 'tit'})]
        titles.extend(temp)
    
        print("page " + str(idx) + "\t: " + str(len(temp)) + " elts")
    
    # add to result
    result.append(titles)
    print('\n\n')

print('finished')

[중고차 다마스/라보] : https://cafe.naver.com/number114?iframe_url=/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=113%26search.boardtype=I%26search.page=
page 1	: 4 elts



[중고차 1톤~1.4톤] : https://cafe.naver.com/number114?iframe_url=/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=14%26search.boardtype=I%26search.page=
page 1	: 20 elts
page 2	: 20 elts
page 3	: 20 elts
page 4	: 20 elts
page 5	: 20 elts
page 6	: 20 elts
page 7	: 20 elts
page 8	: 20 elts
page 9	: 20 elts
page 10	: 20 elts
page 11	: 20 elts
page 12	: 20 elts
page 13	: 20 elts
page 14	: 20 elts
page 15	: 20 elts
page 16	: 13 elts



[중고차 2.5톤~3.5톤] : https://cafe.naver.com/number114?iframe_url=/ArticleList.nhn%3Fsearch.clubid=20319693%26search.menuid=116%26search.boardtype=I%26search.page=
page 1	: 20 elts
page 2	: 20 elts
page 3	: 20 elts
page 4	: 20 elts
page 5	: 20 elts
page 6	: 20 elts
page 7	: 20 elts
page 8	: 20 elts
page 9	: 20 elts
page 10	: 20 elts
page 11	: 20 elts
page 12	: 20 elts
page 13	: 14 elts


In [108]:
# 드라이버 종료
# 새로운 드라이버 실행 전에 반드시 종료해야 함

driver.close()

In [109]:
df_lst = []

for cat, titles in zip(cat_lst, result) :
    df = pd.DataFrame(data=titles, columns=["category", "page", "page_url", "title", "contents_url"])
    df_lst.append(df)
    
result_df=pd.concat(df_lst, axis=0)

In [120]:
result_df.head()

,category,page,page_url,title,contents_url
0,중고차 다마스/라보,1,https://cafe.naver.com/number114?iframe_url=/A...,그랜드 스타렉스 3밴 럭셔리 A/T 무사고(풀옵션) 팝니다,https://cafe.naver.com/number114?iframe_url=/A...
1,중고차 다마스/라보,1,https://cafe.naver.com/number114?iframe_url=/A...,개인용달 다마스 2017년 5월 팝니다,https://cafe.naver.com/number114?iframe_url=/A...
2,중고차 다마스/라보,1,https://cafe.naver.com/number114?iframe_url=/A...,ck미니밴 . 2016년 3월 32000키로주행중,https://cafe.naver.com/number114?iframe_url=/A...
3,중고차 다마스/라보,1,https://cafe.naver.com/number114?iframe_url=/A...,개인용달(라보16년 10월식)3개월 팝니다,https://cafe.naver.com/number114?iframe_url=/A...
0,중고차 1톤~1.4톤,1,https://cafe.naver.com/number114?iframe_url=/A...,1톤봉고 2011년 급매400만원,https://cafe.naver.com/number114?iframe_url=/A...


In [123]:
result_df.to_excel('result.xlsx')